In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用解释性批量预测生成自定义培训表格回归模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> 在Colab中打开
    </a>
  </td>
    <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_custom_tabular_regression_batch_explain.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td> 
     <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_batch_explain.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
        </a>
      </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署一个自定义的表格回归模型，用于批量预测并提供解释。

了解更多关于[Vertex可解释人工智能](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)和[Vertex AI批量预测](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions)的信息。

### 目标

在本教程中，您将学习如何使用Vertex AI训练和可解释AI来创建具有说明的自定义图像分类模型。然后，您将学习如何使用Vertex AI批处理预测服务发出带有解释的批量预测请求。另外，您也可以使用`gcloud`命令行工具或在线使用Cloud Console创建自定义模型。

本教程使用以下Vertex AI服务：

- Vertex AI训练
- Vertex AI批处理预测
- Vertex可解释AI
- Vertex AI模型资源

执行的步骤包括：

- 为训练TensorFlow模型创建Vertex AI自定义作业。
- 查看经过训练的模型的模型评估。
- 设置模型的解释参数。
- 将训练后的模型工件上传为模型资源。
- 进行带有解释的批量预测。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您在本教程中使用的数据集版本是来自于TensorFlow SDK可用的版本。训练好的模型会预测房屋的中位价格，以1K美元为单位。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

开始吧

### 安装 Vertex AI SDK for Python 以及其他所需包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 tensorflow==2.15.1

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等待它完成。⚠️</b>
</div>

请在Google Colab上对您的环境进行身份验证。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type: "string"}

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果你的存储桶还不存在：运行以下单元格来创建你的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import os

import tensorflow as tf
from google.cloud import aiplatform

### 初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须 [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，为了使用一个GPU容器映像，为每个VM分配4个Nvidia Telsa T4 GPU，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

否则，指定`(None, None)`来使用一个在CPU上运行的容器映像。

了解有关[区域硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的更多信息。

**注意**：在支持GPU的TF 2.3之前的TF版本中，无法加载这个教程中的自定义模型。这是一个已知问题，并且在TF 2.3中得到了修复——这是由于生成在serving函数中的静态图操作引起的。如果您在自己的自定义模型上遇到这个问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建Docker容器镜像。

要获取最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

要获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-5"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`以配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU的内存为3.75GB。
     - `n1-highmem`：每个vCPU的内存为6.5GB
     - `n1-highcpu`：每个vCPU的内存为0.9GB
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

**注意**：以下内容不支持用于训练：

 - `标准`：2个vCPUs
 - `高CPU`：2、4和8个vCPUs

**注意**：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 检查培训包

#### 包布局

在开始培训之前，查看Python包是如何为自定义培训工作组装的。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是用于将包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义训练工作的 Python 脚本。

**注意**：在引用工作组规范时，请用点(`trainer.task`)替换目录斜杠，并删除文件后缀 (`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。

总结一下，*task.py* 脚本执行以下操作：

- 从命令行（`--model_dir`）或环境变量 `AIP_MODEL_DIR` 获取要保存模型工件的目录。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用指定的 `args.epochs` 进行训练（`fit()`）。
- 将经过训练的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

请将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成一个压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要完成两个步骤：1）创建一个自定义训练任务，2）运行该任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的人类可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：用于训练容器镜像的包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="boston",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的训练参数

现在为您的自定义训练容器定义命令行参数：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：用于指定存储模型工件位置的命令行参数。您可以使用以下任一方法来指定工件的存储位置。
      - **方法-1**（将`DIRECT`设置为`True`）：您将Cloud Storage位置作为命令行参数传递给您的训练脚本。
      - **方法-2**（将`DIRECT`设置为`False`）：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
  - `--epochs`：训练的epoch数量。
  - `--steps`：每个epoch的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True  # Set False to use AIP_MODEL_DIR
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

运行自定义训练作业

接下来，您可以通过调用`run()`方法来运行自定义作业以启动训练作业，参数如下：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作人员副本的加速器数量。
- `base_output_dir`：存储模型构件的 Cloud Storage 位置。
- `sync`：设置**True**以等待作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在Cloud Storage 存储桶中。现在，从Cloud Storage 存储桶中加载它，然后您可以进行模型评估，并为批量预测请求做准备。

要加载，请使用TF.Keras `model.load_model()`方法，将保存模型的Cloud Storage路径传递给它--由`MODEL_DIR`指定。

In [ ]:
local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在，找出模型的表现如何。为评估准备一些测试数据并运行 `evaluate()` 方法。

### 加载评估数据

使用 `tf.keras.datasets` 中的 `load_data()` 方法加载波士顿房屋测试（留出）数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个元素是测试数据。每个元素又是一个包含两个元素的元组：特征数据和对应的标签（业主占有住房的中位数价值）。

在这一步中，你不需要训练数据。因此，通过加载为 `(_, _)` 来跳过它。

在您可以用数据进行评估之前，您需要对其进行预处理。通过将每个值除以该列的最大值来标准化（重新缩放）每列中的数据。这将用介于 0 和 1 之间的 32 位浮点数替换每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

使用模型的`evaluate()`方法进行评估。

In [ ]:
local_model.evaluate(x_test, y_test)

## 获取服务函数签名

通过将模型重新加载到内存中，并查询每个层对应的签名，您可以获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务函数而不是模型。因此，您需要知道服务函数的输入层名称 - 这在您进行预测请求时会用到。

您还需要知道服务函数的输入和输出层名称，以构建解释元数据 - 这将在接下来讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 说明规范

在进行预测时获取解释时，您必须启用解释功能并在将自定义模型上传到 Vertex AI 模型资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是指定用于在模型上解释的可解释性算法的规范。您可以在以下选项之间进行选择：
  - Shapley（**注意**：不推荐用于图像数据，因为可能运行时间很长）
  - XRAI
  - 集成梯度
- `metadata`：这是指定算法在您的自定义模型上应用的规范。

#### 解释参数

查看解释性算法设置的详细概述。

#### Shapley

为每个特征的结果分配信用，并考虑特征的不同排列。此方法提供了确切 Shapley 值的抽样近似。

用例：
  - 表格数据的分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数量。准确的 Shapley 值近似需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何非常复杂的特征数量，这是非常计算昂贵的。您可以将特征路径数量减少到 M * `path_count`。

#### 集成梯度

一种基于梯度的方法，以高效地计算具有与 Shapley 值相同公理属性的特征指派。

用例：
  - 表格数据的分类和回归。
  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，整体近似度越高。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于集成梯度方法，XRAI 评估图像的重叠区域以创建显著性地图，突显图像的相关区域而非像素。

用例：

  - 图像数据的分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，整体近似度越高。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

在下一个代码单元中，将变量 `XAI` 设置为您在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

#### 说明元数据

现在，让我们深入了解说明元数据，其中包括：

- `outputs`：输出中的标量值，用于属性 -- 要解释的内容。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，希望解释0.7。考虑以下公式，其中输出为`y`，这就是您要解释的内容。

    y = f(x)

考虑下面的公式，其中输出为`y`和`z`。由于您只能对一个标量值进行归因，您必须选择要解释输出`y`还是`z`。假设在此示例中，模型是对象检测，y和z分别是边界框和对象分类。您需要选择要解释其中哪个输出。

    y, z = f(x)

`outputs`字段的字典格式如下：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 - [your_display_name]：您为要解释的输出分配的可读名称。一个常见的示例是"概率"。<br/>
 - "output_tensor_name"：标识要解释的输出层的键/值字段。 <br/>
 - [layer]：要解释的输出层。在单任务模型（例如表格回归器）中，它是模型中的最后（顶部）一层。
</blockquote>

- `inputs`：归因的特征 -- 它们如何影响输出。考虑以下公式，其中`a`和`b`是特征。您必须选择哪些功能来解释它们如何贡献。假设此模型部署用于A/B测试，其中`a`是用于预测的数据项，而`b`标识模型实例是A还是B。您需要选择`a`（或一些子集）作为特征，而不是`b`，因为它不会对预测产生影响。

    y = f(a,b)

`inputs`的最小字典格式如下：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 - [your_display_name]：您为要解释的输入分配的可读名称。一个常见的示例是"特征"。<br/>
 - "input_tensor_name"：用于特征归因的键/值字段。 <br/>
 - [layer]：特征归因的输入层。在单一输入张量模型中，它是模型中的第一个（底部）一层。
</blockquote>

由于模型的输入为表格型数据，您可以指定以下两个附加字段作为报告/可视化辅助工具：

<blockquote>
 - "modality": "image": 表示字段值为图像数据。
</blockquote>

由于模型的输入为表格型数据，您可以指定以下两个附加字段作为报告/可视化辅助工具：

<blockquote>
 - "encoding": "BAG_OF_FEATURES"：表示输入为一组表格特征。<br/>
 - "index_feature_mapping": [feature-names]：每个功能的人类可读名称的列表。在此示例中，您使用数据集中指定的特征名称。<br/>
 - "modality": "numeric"：表示字段值为数字。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aiplatform.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aiplatform.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aiplatform.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到Vertex AI模型注册表，使用以下参数：

- `display_name`：模型资源的可读名称。
- `artifact`：经过训练的模型工件的Cloud Storage位置。
- `serving_container_image_uri`：用于提供服务的容器镜像。
- `sync`：是否异步或同步执行上传。
- `explanation_parameters`：用于配置解释模型预测的参数。
- `explanation_metadata`：描述解释模型的输入和输出的元数据。

如果`upload()`方法是异步运行的，您可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aiplatform.Model.upload(
    display_name="boston",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

发送批量预测请求

创建测试项目作为您的批量预测请求的输入。然后，向部署的模型发送批量预测请求。

### 制作测试项目

您可以使用预处理后的评估数据（x_test）的一部分来进行批量请求。

### 制作批处理输入文件

现在创建一个批处理输入文件，并将其存储在本地的云存储桶中。与图像、视频和文本不同，表格的批处理输入文件只支持 CSV 格式。对于 CSV 文件，您应该：

- 第一行是包含特征（字段）名称的标题。
- 每个剩余的行是一个独立的预测请求，带有相应的特征值。

例如：

    "feature_1", "feature_2". ...
    value_1, value_2, ...

In [ ]:
with open("batch.csv", "w") as f:
    f.write("crim, zn, indus, chas, nox, rm, age, dis, rad, tax, ptratio, b, lstat\n")
    f.write(str(x_test[0].tolist()).replace("[", "").replace("]", ""))
    f.write("\n")
    f.write(str(x_test[1].tolist()).replace("[", "").replace("]", ""))
    f.write("\n")

gcs_input_uri = BUCKET_URI + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 发送批量解释请求

现在您的模型资源已经训练完成，您可以通过调用`batch_predict()`方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `generate_explanations`：设置为`True`以生成解释。
- `sync`：如果设置为`True`，则调用在等待异步批量任务完成时会阻塞。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="boston",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批量作业完成。或者，您可以在`batch_predict()`方法中将`sync`参数设置为`True`，以等待批量预测作业完成。

In [ ]:
batch_predict_job.wait()

获取解释

接下来，从完成的批量预测作业中获取解释结果。

结果将写入您在批量预测请求中指定的Cloud Storage输出存储桶。您可以调用`iter_outputs()`方法获取生成结果的每个Cloud Storage文件的列表。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation"):
        explanation_results.append(blob.name)

tags = list()
for explanation_result in explanation_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Delete the model
model.delete()

# Delete the batch prediction job
batch_predict_job.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Remove the locally generated files
! rm -rf batch.csv custom.tar.gz custom